04/14/2022

In [2]:
!pip install pyqubo
!pip install docplex
!pip install cplex

     |████████████████████████████████| 229 kB 9.9 MB/s 
     |████████████████████████████████| 497 kB 48.0 MB/s 
     |████████████████████████████████| 13.2 MB 26.2 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.8
    Uninstalling pyparsing-3.0.8:
      Successfully uninstalled pyparsing-3.0.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 610 kB 6.8 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=18e4907990e954d3a2ca4ae50505f0ec7f9420ece5bfbf56d1baf9872237273c
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex
     |████████████████████████████████| 43.3 MB 1.2 MB/s 


In [3]:
from pyqubo import Array
import numpy as np
import math
import neal
import time
import docplex.mp.model as md

In [4]:
np.random.seed(87)
n=5
numbers=np.random.randint(1,100,n)

# Normalize Data
numbers_norm= (numbers)/(np.max(numbers))
print('Numbers: ', numbers)

Numbers:  [78 60 16  7  7]


Objective: Order the Numbers. The Hamiltonian Should correspond to a linear sequence of numbers with just as many spins as numbers such that
$$H_1 = - \sum_i^N s_i n_i $$ where $N$ is the quanity of numbers we are ordering. The largest spin will fall with the largest number it can find. We also add the constraint that no spin can be used more than once for $N$ spins and $N$ numbers (need to add constraint in H for distinct spins on each number). We could add $$ H_2 = \left| \sum_i s_i - \frac{(n-1)n}{2} \right|*\alpha$$. We can then take into account the prime factorization of the product, which must be unique
$$ H_3 = \left| \prod_{i=1}^n s_i - n!\right|*\beta$$ where $\beta$ is just some weighted coefficient. Therefore, our hamiltonian becomes
$$ H = H_1 + H_2 + H_3$$.

CPLEX

In [7]:
start = time.time()
model=md.Model(name='Number_Ordering')
s=model.integer_var_list(numbers_norm.shape[0],lb=1,ub=n,name='order')
model.add_constraints_(s[i]!=s[j] for i in range(len(s)) for j in range(len(s)) if i!=j )
H1=np.sum((-1)*numbers_norm*s)
H2=1-2*np.sum(s)/n/(n+1) #needs an np.abs() around this term to catch 0, however, the model 
# dislikes this greatly
#H3= np.prod(s)-np.math.factorial(n)  model cannot handle a polynomial of order >=3, only up to degree 2
model.minimize( H1+H2**2)
solve_model = model.solve(url=None,key=None)
solution=np.array(model.solution.get_value_list(s),dtype=int)
time_cplex=time.time()-start

print('Original Numbers: ', numbers)
print('Ordered Index: ', solution)
ordered_list=np.empty(n)
for i in range(len(solution)):
  ordered_list[solution[i]-1]=numbers[i]
print('Ordered_List: ', ordered_list)
print('Total Time:' , time_cplex,'s')

Original Numbers:  [78 60 16  7  7]
Ordered Index:  [5 4 3 2 1]
Ordered_List:  [ 7.  7. 16. 60. 78.]
Total Time: 0.0785074234008789 s
